In [1]:
import os
import glob
import cv2
import numpy as np
import easyocr
import Utils
from cityCode import cityCode_dict
from datetime import datetime

imagesPath = "Images\\"
reader = easyocr.Reader(['en'])
numberPlatePoints = []

heightNumberPlate = 250

def main():
    for filename in glob.glob(imagesPath+"*.jpg"):
        img = cv2.imread(filename)
        scale = round(heightNumberPlate / img.shape[0], 1)
        img = Utils.resize(img, scale)

        detail = reader.readtext(img)

        numberPlate, datePlate = check1Line(img, detail)
        print("numberPlate:", numberPlate)
        print("datePlate:", datePlate)
        print("---"*20)

        numberPlate, code, city, oddEven = detailNumberPlate(numberPlate)
        datePlate, month, year = detailDatePlate(datePlate)

        printDetail(numberPlate, datePlate, code, city, oddEven, month, year)
        
        img = showText(img, numberPlate,datePlate,code,city,oddEven,month,year)

        cv2.imshow("img", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def showText(img, numberPlate,datePlate,code,city,oddEven,month,year):
    color = (255,255,0)
    colorG = (0,255,0)
    colorR = (0,0,255)
    thickness = 2
    scale = 2
    distance = int(img.shape[0] / 7)

    currentDay = datetime.now().day
    currentMonth = datetime.now().month
    currentYear = datetime.now().year
    timeNow = str(currentDay) + "/" + str(currentMonth) + "/" + str(currentYear)

    status = ""
    if currentDay%2 == 0:
        status = "Even"
    else:
        status = "Odd"
    statusText = "Status: " + status
    #Black
    img[0:int(1 * distance)+5 , img.shape[1] - 170:img.shape[1]] = np.zeros(((1*distance)+5 , 170, 3), np.uint8)
    
    cautionStatus = ""
    print(status, oddEven)
    if status != oddEven:
        cautionStatus = " (VIOLATION!)"    

    cautionExpired = ""
    if year < currentYear:
        cautionExpired = " (VIOLATION!)"
    elif year == currentYear:
        if month <= currentMonth:
            cautionExpired = " (VIOLATION!)"
    textExpired = "Expired: " + str(month) + "/" + str(year)

    cv2.putText(img, timeNow, (img.shape[1] - 170, int(0.5 * distance)), cv2.FONT_HERSHEY_PLAIN, 1.5, color, thickness)
    cv2.putText(img, statusText, (img.shape[1] - 170, int(1 * distance)), cv2.FONT_HERSHEY_PLAIN, 1.5, color, thickness)

    cv2.putText(img, numberPlate, (20, img.shape[0] - int(2.1 * distance)), cv2.FONT_HERSHEY_PLAIN, scale, color, thickness)
    cv2.putText(img, "City: " + str(city), (20, img.shape[0] - int(1.5 * distance)), cv2.FONT_HERSHEY_PLAIN, 1.5, colorG, thickness)
    cv2.putText(img, "Type: " + oddEven, (20, img.shape[0] - 1 * distance), cv2.FONT_HERSHEY_PLAIN, 1.5, colorG, thickness)
    cv2.putText(img, textExpired, (20, img.shape[0] - int(0.5 * distance)), cv2.FONT_HERSHEY_PLAIN, 1.5, colorG, thickness)
    cv2.putText(img, cautionStatus, (20+135, img.shape[0] - int(1 * distance)), cv2.FONT_HERSHEY_PLAIN, 1, colorR, thickness)
    cv2.putText(img, cautionExpired, (20+225, img.shape[0] - int(0.5 * distance)), cv2.FONT_HERSHEY_PLAIN, 1, colorR, 2)

    return img


def printDetail(numberPlate, datePlate, code, city, oddEven, month, year):
    print("numberPlateFix:", numberPlate)
    print("datePlateFix:", datePlate)
    print("- city code:", code)
    print("- city:", city)
    print("- type:", oddEven)
    print("- month:", month)
    print("- year:", year)
    print("---"*20)

def detailDatePlate(datePlate):
    datePlate = correctNumber(datePlate)

    month = int(datePlate[:2])
    year = int("20" + datePlate[-2:])
    return datePlate, month, year

def correctNumber(datePlate):
    datePlate = list(datePlate)
    for i in range (len(datePlate)):

        if datePlate[i] == "D" or datePlate[i] == "O" or datePlate[i] == "o":
            datePlate[i] = "0"
        elif datePlate[i] == "S" or datePlate[i] == "s":
            datePlate[i] = "5"
        elif datePlate[i] == "G":
            datePlate[i] = "6" 
        elif datePlate[i] == "B":
            datePlate[i] = "8" 

    datePlate = "".join(datePlate)
    datePlate = datePlate.replace(" ", "")

    #for r in ((".", " "), ("*", " "), (",", " "), ("'", " ")):
        #datePlate = datePlate.replace(*r)

    return datePlate    

def detailNumberPlate(numberPlate):

    numberPlate = corrrectCity(numberPlate)
    numberPlate = correctCapital(numberPlate)

    code, city = checkCity(numberPlate)
    oddEven = checkOddEven(numberPlate)

    return numberPlate, code, city, oddEven

def checkOddEven(numberPlate):
    status = "None"
    for x in numberPlate:
        try:
            x = int(x)
            if x%2 == 0:
                status = "Even"
            else:
                status = "Odd"
        except ValueError:
            pass

    return status

def correctCapital(numberPlate):
    numberPlate = list(numberPlate)
    numberPlate = [x.upper() for x in numberPlate]
    numberPlate = "".join(numberPlate)

    return numberPlate

def checkCity(numberPlate):
    cityCode = numberPlate.split()[0]

    if cityCode in cityCode_dict:
        city = cityCode_dict[cityCode]
    else:
        city = "Error City"

    return cityCode, city

def corrrectCity(numberPlatePoints):

    numberPlatePoints = list(numberPlatePoints)
    for i in range (2): # 2 First Digit

        if numberPlatePoints[i] == "0":
            numberPlatePoints[i] = "D"
        elif numberPlatePoints[i] == "5":
            numberPlatePoints[i] = "S"
        elif numberPlatePoints[i] == "6":
            numberPlatePoints[i] = "G" 
        elif numberPlatePoints[i] == "8":
            numberPlatePoints[i] = "B" 

    numberPlatePoints = "".join(numberPlatePoints)
    return numberPlatePoints

def check1Line(img, detail):
    date = False
    tolerance = int (img.shape[0] / 7)
    yline1 = 0
    numberPlate = ""

    datePlate = ""

    for i in range (len(detail)):        
        # Check Line
        if i == 0:
            yline1 = detail[i][0][0][1]
            numberPlate = detail[i][1]
            
        else:
            if date == False:
                yMin = yline1 - tolerance
                yMax = yline1 + tolerance
                if yMin < detail[i][0][0][1] < yMax:
                    numberPlate = numberPlate + " " + detail[i][1]
                else:
                    date = True

        if date ==  True:
            datePlate = datePlate + detail[i][1]

        img = createRectangle(img, detail[i][0])

    return numberPlate, datePlate

def createRectangle(img, points):
    color = (0,255,255)
    thickness = 1
    cv2.line(img, points[0], points[1], color, thickness)
    cv2.line(img, points[1], points[2], color, thickness)
    cv2.line(img, points[2], points[3], color, thickness)
    cv2.line(img, points[3], points[0], color, thickness)

    return img

if __name__ == '__main__':       
    # Calling main() function 
    main()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
C:\Users\hanif\AppData\Roaming\Python\Python39\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


numberPlate: AA L337 H
datePlate: 12.25
------------------------------------------------------------
numberPlateFix: AA L337 H
datePlateFix: 12.25
- city code: AA
- city: ['Purworejo', 'Tamanggung', 'Magelang', 'Wonosobo', 'Kebumen']
- type: Odd
- month: 12
- year: 2025
------------------------------------------------------------
Even Odd
numberPlate: AG 3738 IU
datePlate: 0420
------------------------------------------------------------
numberPlateFix: AG 3738 IU
datePlateFix: 0420
- city code: AG
- city: ['Tulungagung', 'Kediri', 'Blitar', 'Trenggalek']
- type: Even
- month: 4
- year: 2020
------------------------------------------------------------
Even Even
numberPlate: 8 1991 JO
datePlate: 07 . 17
------------------------------------------------------------
numberPlateFix: B 1991 JO
datePlateFix: 07.17
- city code: B
- city: ['DKI Jakarta', 'Bekasi', 'Depok']
- type: Odd
- month: 7
- year: 2017
------------------------------------------------------------
Even Odd
numberPlate: BA 1

In [3]:
a_string = "aba"
print(a_string)
a_string = a_string.replace("a", "b")

print(a_string)


aba
bbb
